In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report
from tensorflow.keras.models import load_model
 

warnings.filterwarnings("ignore")

try:
    # Read the CSV file
    df = pd.read_csv("/kaggle/input/real-or-fake-fake-jobposting-prediction/fake_job_postings.csv")

    # Drop duplicate values
    df.drop_duplicates(inplace=True)

    # Drop columns with null values more than 60%
    missing_values = df.columns[((df.isnull().sum() / len(df.index)) * 100) > 60]
    df.drop(missing_values, axis=1, inplace=True)
    df.drop("job_id", axis=1, inplace=True)

    # Separate input and output variables
    x = df.iloc[:, :-1]
    y = df.iloc[:, -1]

    # Fill null values in categorical columns with mode
    x_cat = x.select_dtypes(object)
    x_cat = x_cat.fillna(x_cat.mode().iloc[0])

    # Apply one-hot encoding to categorical features
    x_cat = pd.get_dummies(x_cat)

    # Apply standard scaler on numeric data
    x_num = x.select_dtypes(['int64', 'float64'])
    ss = StandardScaler()
    x_num = pd.DataFrame(ss.fit_transform(x_num))

    # Concatenate categorical and numeric data
    x = pd.concat([x_num, x_cat], axis=1)

    # Encode output variable
    le = LabelEncoder()
    y = pd.DataFrame(le.fit_transform(y))

    # Remove rows with missing values
    xy = pd.concat([x, y], axis=1).dropna()
    x = xy.iloc[:, :-1]
    y = xy.iloc[:, -1]

    # Reindex x and y
    x.reset_index(drop=True, inplace=True)
    y.reset_index(drop=True, inplace=True)

    # Split train and test data
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1)

    # Create and compile the model
    ann = Sequential()
    ann.add(Dense(16, activation="relu", input_shape=(x.shape[1],)))
    ann.add(Dropout(rate=0.2))
    ann.add(Dense(8, activation="relu"))
    ann.add(Dropout(rate=0.2))
    ann.add(Dense(units=1, activation="sigmoid"))
    ann.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

    # Train the model
    ann.fit(x_train, y_train, epochs=30, batch_size=20)

    # Generate predictions
    y_pred = ann.predict(x_test)
    y_pred = np.round(y_pred).astype(int)

    # Print classification report
    print(classification_report(y_test, y_pred))

    # Calculate accuracy
    accuracy = (y_test.values == y_pred).mean() * 100
    print(f"Accuracy: {accuracy}")

    # Save the model
    ann.save("fakejobposting_model")
    storemodel = load_model("fakejobposting_model")

except KeyboardInterrupt:
    print("Program terminated by user.")

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

Epoch 1/30
626/626 [==============================] - 5s 7ms/step - loss: 0.1765 - accuracy: 0.9495
Epoch 2/30
626/626 [==============================] - 5s 7ms/step - loss: 0.0652 - accuracy: 0.9740
Epoch 3/30
626/626 [==============================] - 5s 8ms/step - loss: 0.0237 - accuracy: 0.9914
Epoch 4/30
626/626 [==============================] - 4s 7ms/step - loss: 0.0116 - accuracy: 0.9974
Epoch 5/30
626/626 [==============================] - 4s 7ms/step - loss: 0.0086 - accuracy: 0.9990
Epoch 6/30
626/626 [==============================] - 4s 7ms/step - loss: 0.0067 - accuracy: 0.9998
Epoch 7/30
626/626 [==============================] - 4s 7ms/step - loss: 0.0056 - accuracy: 1.0000
Epoch 8/30
626/626 [==============================] - 4s 7ms/step - loss: 0.0055 - accuracy: 0.9998
Epoch 9/30
626/626 [==============================] - 4s 7ms/step - loss: 0.0044 - accuracy: 0.9998
Epoch 10/30
626/626 [==============================] - 4s 7ms/step - loss: 0.0033 - accuracy: 0.9999